In [ ]:
###
###CAMERA_CALIBRATION
###
import os
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import glob
img_chessboard = glob.glob('camera_cal/calibration*.jpg')
# prepare object points
nx = 9# number of inside corners in x
ny = 6# number of inside corners in y
objpoints = [] # 3d points 
imgpoints = [] # 2d points 
###this code is the same as those appears in lecture video
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
for index, filename in enumerate(img_chessboard):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

    
    # If found, draw corners    
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        #plt.imshow(img)

#get mtx and dist
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
print("Get mtx and dist")


In [ ]:
###
###UNDISTORT IMAGE
###
#undistort image
# dst = cv2.undistort(img, mtx, dist, None, mtx)
def undist(img,mtx,dist):
    return cv2.undistort(img, mtx, dist, None, mtx)
print("undistort function was built")


###undist visulization
##code below from correct for distortion lecture
img= mpimg.imread("camera_cal/calibration1.jpg")
undistorted = undist(img, mtx, dist)
###
###visualization pipline
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
###
#COLOR AND GRADIENT THRESHOLD
###

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold

    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    grad_binary = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    grad_binary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return grad_binary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    # 3) Calculate the magnitude 
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    grad = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    # 5) Create a binary mask where direction thresholds are met
    # 6) Return this mask as your binary_output image
    binary_output = np.zeros_like(grad)
    #binary_output = np.copy(img) # Remove this line
    ##apply the threshold
    binary_output[(grad >= thresh[0]) & (grad <= thresh[1])] = 1
    return binary_output

def color_threshold(s_channel, thresh=(90, 255)):
    hls = cv2.cvtColor(warped, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    binary = np.zeros_like(S)
    binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary




print("threshold pipline was built")


In [ ]:
def pipeline(img, s_thresh=(90, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    h_channel = hsv[:,:,0]
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    return color_binary
    
print("build pipline")

In [ ]:
###
##read test image
###
img_test = glob.glob('test_images/test*.jpg')
sample=[]
for index, filename in enumerate(img_test):
    img = cv2.imread(filename)
    sample.append(img)
    
# test_images = []
# for file in os.listdir('test_images/'):
#     if(file != '.DS_Store'):
#         test_images.append("test_images/"+file)
        
print("test images loaded")

In [ ]:
###
##test color threshold
###

    
case1 = sample[2]
undistortion = undist(case1, mtx, dist)
warped = undistortion.copy()


case_Result = color_threshold(warped, thresh=(90, 255))
# result = pipeline(warped)

##plot case result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(warped)
ax1.set_title('warped Image', fontsize=50)
ax2.imshow(case_Result, cmap='gray')
ax2.set_title('Thresholded S', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
###
##test threshold
###

    
case2 = sample[1]
undistortion2 = undist(case2, mtx, dist)
warped_2 = undistortion2.copy()



# case_Result = color_threshold(warped, thresh=(90, 255))
result = pipeline(warped_2)

##plot case result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(warped_2)
ax1.set_title('warped Image', fontsize=50)
ax2.imshow(result, cmap='gray')
ax2.set_title('Thresholded S and Sobel', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
#Perspective Transform




# def image_unwarp(undisto,img_size):

#     left_top_src = [550, 580]
#     left_down_src =[800, 580]
#     right_top_src = [400, 680] 
#     right_down_src = [1050, 680]
#     src = np.array([ left_top_src,left_down_src,right_down_src,right_top_src], dtype=np.float32)

#     left_top_dst = [300, 310]
#     left_down_dst = [700, 370]
#     right_top_dst = [300, 870]
#     right_down_dst = [700, 800]
#     dst = np.array([left_top_dst,left_down_dst,right_down_dst,right_top_dst], dtype=np.float32)
    
#     M = cv2.getPerspectiveTransform(src, dst)
#         # Warp the image using OpenCV warpPerspective()
#     warped_img = cv2.warpPerspective(undisto, M, img_size)
#     #warped_img = cv2.transpose(warped_img)
#     return warped_img, M

####this warper function is copied from github
##my warper did not work
def image_unwarp(img, bird_view = True):
    xsize = img.shape[1]
    ysize = img.shape[0]

    p1_src = [175, ysize]
    p2_src = [560, 470]
    p3_src = [730, 470]
    p4_src = [1155, ysize]
    src = np.array([p1_src, p2_src, p3_src, p4_src], dtype=np.float32)

    p1_dst = [250, ysize]
    p2_dst = [250, 0]
    p3_dst = [1030, 0]
    p4_dst = [1030, ysize]
    dst = np.array([p1_dst, p2_dst, p3_dst, p4_dst], dtype=np.float32)
    
    if(bird_view):
        # if we need to change to bird view 
        # given src and dst points, calculate the perspective transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
    else:
        # else switch src and dst, change back from bird view
        M = cv2.getPerspectiveTransform(dst, src)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), cv2.INTER_LINEAR)
    return warped, M
    

##test perspective transform

gray_case = cv2.cvtColor(warped,cv2.COLOR_BGR2GRAY)
img_size=(gray_case.shape[1],img.shape[0])


top_down, M= image_unwarp(undistortion)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(warped)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(top_down)
ax2.set_title('Undistorted and Warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
import numpy as np
s_img = top_down[:,:,2]
histogram2 = np.sum(top_down[top_down.shape[0]/2:,:], axis=0)
plt.plot(histogram2)

In [ ]:


###plot S channel

histogram = np.sum(s_img[s_img.shape[0]/2:,:], axis=0)
plt.plot(histogram)

In [ ]:
###find lane line 
import fit_polynomial

gray_lane = cv2.cvtColor(top_down,cv2.COLOR_BGR2GRAY)
ploty, left_fitx, right_fitx, leftx,rightx= fit_polynomial.fit_polynomial(gray_lane)


In [ ]:
###
#measure curvature
###
mark_size = 3
# leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
# rightx = rightx[::-1] 

# plt.plot(leftx, ploty, 'o', color='red', markersize=mark_size)
# plt.plot(rightx, ploty, 'o', color='blue', markersize=mark_size)
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.plot(left_fitx, ploty, color='green', linewidth=3)
plt.plot(right_fitx, ploty, color='green', linewidth=3)
plt.gca().invert_yaxis() # to visualize as we do the images

In [ ]:
import silding_window_search
gray_lane = cv2.cvtColor(top_down,cv2.COLOR_BGR2GRAY)
window = silding_window_search.get_window_centroids_result(gray_lane)

In [ ]:
###
#Draw lane line 
###
warp_zero = np.zeros_like(case_Result).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space using inverse perspective matrix (Minv)
# newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
# gray_topdown = cv2.cvtColor(top_down,cv2.COLOR_BGR2GRAY)
# img_size=(gray_topdown.shape[1],img.shape[0])
# Combine the result with the original image
newwarp, M= image_unwarp(color_warp,bird_view=False)
_result = cv2.addWeighted(undistortion, 1, newwarp, 0.3, 0)
plt.imshow(_result)

print("get lane line")

In [ ]:
#####
##run the video
###
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(_img):
    undist_v = undist(_img,mtx,dist)
    V_Result = color_threshold(undist_v, thresh=(90, 255))
    w_img, M= image_unwarp(undist_v)
    
    _lane = cv2.cvtColor(w_img,cv2.COLOR_BGR2GRAY)
    ploty, left_fitx, right_fitx, leftx,rightx= fit_polynomial.fit_polynomial(_lane)
    
    warp_z = np.zeros_like(V_Result).astype(np.uint8)
    color_w = np.dstack((warp_z, warp_z, warp_z))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_w, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    # newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # gray_topdown = cv2.cvtColor(top_down,cv2.COLOR_BGR2GRAY)
    # img_size=(gray_topdown.shape[1],img.shape[0])
    # Combine the result with the original image
    newwarp, M= image_unwarp(color_warp,bird_view=False)
    _result = cv2.addWeighted(undist_v, 1, newwarp, 0.3, 0)
    return _result
print("process image function")

In [ ]:
video_output = '_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(video_output, audio=False)

VIDEO RESULT:

PRETTY GOOD! You can get my test result in _output.mp4

In [ ]:
##proc all test images
test_images = []
for file in os.listdir('test_images/'):
    if(file != '.DS_Store'):
        test_images.append("test_images/"+file)
print("read all test file")
prefix = 'output_images/' 
im_index = 0
for t_file in  test_images:
    
    t_img = cv2.imread(t_file)
    
    t_result = process_image(t_img)
    im_index = im_index+1
    cv2.imwrite(prefix+str(im_index)+".jpg",t_result)
print("Done")

Test images' output can be found in output_images folder